## Load the IMDB dataset and create the vocabulary

In [1]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from collections import Counter, OrderedDict
from torchtext.vocab import vocab

EMBEDDING_DIM=50
VOCAB_SIZE=20000


# Get IMDB dataset
imdb = IMDB(split='train')

# Load English tokenizer, tagger, parser and NER
tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

# build the vocab
counter = Counter()
for i, (label, line) in enumerate(imdb):
    counter.update(tokenizer(line))

ordered_dict = OrderedDict(counter.most_common()[:VOCAB_SIZE])
vocab = vocab(ordered_dict)

# insert special tokens and set default index to 'unknown'
vocab.insert_token('<PAD>', 0)
vocab.insert_token('<UNK>', 1)
vocab.set_default_index(1)

## Create embedding vectors from GloVe

In [2]:
import torchtext as text

# load glove embeddings
vec = text.vocab.GloVe(name='6B', dim=50)
# create the embedding matrix, a torch tensor in the shape (num_words+1, embedding_dim)
word_emb = vec.get_vecs_by_tokens(vocab.get_itos())

In [3]:
BATCH_SIZE = 32

## Build up train/test dataset

In [4]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# transform input text and label to ids
def process_text(text):
    return vocab(tokenizer(text))

label_to_ids = {'pos':0, 'neg':1}

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for (_label, _text) in batch:
        label_list.append(label_to_ids[_label])
        processed_text = torch.tensor(process_text(_text), dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
        
    # label must be in the same size as target
    label_list = torch.tensor(label_list, dtype=torch.float)[:,None]

    # Creating mask 
    # print(word_emb.size())
    # print(type(lengths))
    # print(len(lengths))
    # print(lengths)
    # print('max', max(lengths))
    last_indices = torch.sub(torch.LongTensor(lengths), 1)
    mask = torch.zeros(BATCH_SIZE, max(lengths), 1)

    rows = torch.arange(0, BATCH_SIZE).long()
    mask[rows, last_indices, :] = 1

    text_list = pad_sequence(text_list, batch_first=True)
    lengths = torch.tensor(lengths, dtype=torch.float)
    return label_list.to(device), text_list.to(device), lengths.to(device), mask.to(device)

train_iter, test_iter = IMDB()

# transform datasets iterator into map style so that they can be repeatedly loaded in a new epoch
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

In [8]:
# logistic model
import torch
import torch.nn as nn
import torch.nn.functional as F

class LogisticRegression(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        self.fc = nn.Linear(embed_dim, 1)
        self._init_weights()

    def _init_weights(self):
        """Initialize network parameters 
        """
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, lengths):
        embedded = self.embedding(text) # (batch_size, sent_len, emb_size)
        embedded = embedded.sum(dim = 1) / lengths[:, None] # (add one axis)
        return torch.sigmoid(self.fc(embedded))

class LSTMcustom(nn.Module):
    def __init__(self, word_vec, embed_dim):
        super().__init__()
        # embeddingbag outputs the average of all the words in a sentence
        self.embedding = nn.Embedding(*(word_vec.size())).from_pretrained(word_vec, freeze=False)
        
        # Initialize LSTM model. The arguments are in this order input_dim, hidden_dim, n_layers
        self.lstm = nn.LSTM(embed_dim, 200, 1, bidirectional=False, batch_first = True)

        # Adding multiple layers to LSTM (Q4)
        self.lstm_2layer = nn.LSTM(embed_dim, 200, 2, bidirectional=False, batch_first = True)
        self.lstm_3layer = nn.LSTM(embed_dim, 200, 3, bidirectional=False, batch_first = True)
        self.lstm_4layer = nn.LSTM(embed_dim, 200, 4, bidirectional=False, batch_first = True)

        # Simple RNN layer (Q5)
        self.rnn_tanh = nn.RNN(embed_dim, 200, 1, nonlinearity='tanh', bidirectional=False, batch_first = True)
        self.rnn_relu = nn.RNN(embed_dim, 200, 1, nonlinearity='relu', bidirectional=False, batch_first = True)

        # Q1: What should be the input dimension of this linear layer?
        self.fc = nn.Linear(200, 1)
        
        # Q2: set bidirectional = True in the LSTM and determine the input size of the FC layer
        self.lstm_bi = nn.LSTM(embed_dim, 200, 1, bidirectional=True, batch_first = True)
        self.fc_bi = nn.Linear(200 * 2, 1)
        
    def forward(self, text, lengths, mask):
        embedded = self.embedding(text) # (batch_size, sent_len, emb_size)
        
        lstm_out,_ = self.lstm(embedded) # lstm_out is a 3d tensor (batch_size, sent_len, output_size). If you have a bidirectional LSTM, the outputsize will be 2*output_size
        # print('lstm-out', lstm_out.size(1))
        # print(2, word_vec.size())
        # Q3: Select the hidden output of the last element in the sequence. Hint: Remember that you padded the sequence and you already know the length of the sequence.
        mask = mask.expand(-1,-1,200) # 200 * 2 for biderectional LSTM
        product = lstm_out * mask
        last_states = torch.sum(product, dim=0).long()

        lstm_out = lstm_out[:][last_states][:] #
        
        # Q4: Add multiple layers to the LSTM by passing argument num_layers=2,3,4 etc. to the LSTM function above. Test whether the performance improves. If not, explain what did you observe.
        # Added below the LSTM definition above. 

        # Q5: Replace the LSTM layer with simple RNN layer and compare the performance between LSTM and RNN.
        return torch.sigmoid(self.fc(lstm_out))   

In [6]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, lengths, mask) in enumerate(dataloader):
        optimizer.zero_grad()
        # forward propagation
        predicted_label = model(text, lengths,mask)
        # calculate loss and backpropagate to model paramters
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # update parameters by stepping the optimizer
        optimizer.step()
        total_acc += ((predicted_label > 0.5) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, lengths,mask) in enumerate(dataloader):
            predicted_label = model(text, lengths,mask)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
# Hyperparameters
EPOCHS = 2 # epoch

model = LSTMcustom(word_vec=word_emb, embed_dim=EMBEDDING_DIM).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()
total_accu = None


for epoch in range(1, EPOCHS + 1):
    print("LSTM with 1 layer")
    epoch_start_time = time.time()
    train(train_dataloader)

In [ ]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.2f}%'.format(accu_test))

In [10]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
!kill -9 17952
!nvidia-smi
